## AI Assignment 2

In [1]:
# Imports
from tkinter import messagebox,Label
import tkinter as tk
import numpy as np
import time

## Class of robotic System that contains all functions and its implementation 

In [2]:
class roboticSystem:   
    def __init__(self):
        self.Robot=None
        self.cells=None
        self.EmptyRobotArm=True
        self.cellsConnections=None
        self.objects= []
        self.objectsAfterReachGoal=[]
        self.goals=[]
        self.obstacles=[]
        self.draw=True
        
    # Function to build grid , it requires number of cols and rows
    def buildCells(self,c,rows,cols):
        cells = [[None for _ in range(cols)] for _ in range(rows)]
        # Get rectangle diameters by dividing the width and height of canvas over number of cells
        col_width = int(c.winfo_width()/cols)
        row_height = int(c.winfo_height()/rows)
        # Loop to create each cell
        for x in range(rows):
            for y in range (cols):
                cells[x][y] = c.create_rectangle((y)*col_width, (x)*row_height, (y+1)*col_width, \
                                                     (x+1)*row_height, outline='black')
        # Returning the ids of created cells
        x=np.array(x).flatten()
        self.cells= np.array(cells).flatten()
        
        

    # A created function that interputs the grid   
    def middle(self,event,c):
         # The function checks with button to be pressed to go to change_color Fucntion
        letter=event.char
        c.bind("<Button-1>",lambda e: self.change_color(event=e,c=c,char=letter))
        
        

    
    #Function that changes color of the cell
    def change_color(self,event,c,char):
        if self.draw==False:
            return
        # Getting the object at pressed location
        items=c.find_closest(event.x, event.y)
        # Checking which key buttion is cliked to color the object
        if items and char=='r' :
            # Checking if the robot if defined
            if(self.Robot==None):
                #Checking if the cell that robot want to occuipy is already selected or no
                if (items[0] in self.objects or items[0] in self.obstacles or items[0]in self.goals):
                    msg='You choosed an occuiped cell delete it first'
                    messagebox.showinfo("Cell Alert", msg)
                    return
                # Then selecting the cell with robot state and save the location of it
                c.itemconfigure(items, fill="green")
                self.Robot=items[0]
            else :
                messagebox.showinfo("Robot Cell Alert", "You already choosed an robot cell")
        #Selecting the blocks
        elif items and char=='b':
            #Checking if the cell that obstacles want to occuipy is already selected or no
            if (items[0] in self.objects or items[0] in self.obstacles or items[0]in self.goals or items[0] == self.Robot ):
                msg='You choosed an occuiped cell delete it first'
                messagebox.showinfo("Cell Alert", msg)
                return
            # Then selecting the cell to be a block and save the location of it
            c.itemconfigure(items, fill="gray")
            self.obstacles.append(items[0])
        # Selecting the objects    
        elif items and char=='o':
            #Checking if the cell that objects want to occuipy is already selected or no
            if (items[0] in self.objects or items[0] in self.obstacles or items[0]in self.goals or items[0] == self.Robot ):
                msg='You choosed an occuiped cell delete it first'
                messagebox.showinfo("Cell Alert", msg)
                return
            c.itemconfigure(items, fill="blueviolet")
            self.objects.append(items[0])
        # Selecting the goals        
        elif items and char=='g':
            #Checking if the cell that goals want to occuipy is already selected or no
            if (items[0] in self.objects or items[0] in self.obstacles or items[0]in self.goals or items[0] == self.Robot ):
                msg='You choosed an occuiped cell delete it first'
                messagebox.showinfo("Cell Alert", msg)
                return
            c.itemconfigure(items, fill="gold")
            self.goals.append(items[0])  
        # Defining how to delete occuiped cell if we want, we press D and select the cell to delete    
        elif items and char=='d':
            if(self.Robot==items[0]):
                c.itemconfigure(self.Robot, fill="white")
                self.Robot=None
            elif(any([True for x in self.objects if items[0]==x])):
                c.itemconfigure(items[0], fill="white")
                self.objects.remove(items[0])
            elif(any([True for x in self.goals if items[0]==x])):
                c.itemconfigure(items[0], fill="white")
                self.goals.remove(items[0])
            elif(any([True for x in self.obstacles if items[0]==x])):
                c.itemconfigure(items[0], fill="white")
                self.obstacles.remove(items[0])
            else:
                c.itemconfigure(items[0], fill="white")
    
    def connectEdge(self,key,value,rows,cols):
        # Just adding conditions not to get out of grid borderes
        if value<=0 or (key%cols==1 and (key-value)==1) or(key%cols==0 and (value-key)==1) or value>=121:
            return
        #Adding cells that the obstacles from connections 
        if value in self.obstacles or value in self.objectsAfterReachGoal:
            return
        if self.cellsConnections[key-1] != None:
            edgeList = self.cellsConnections[key-1]
            edgeList.append(value)
        else:
            edgeList = []
            edgeList.append(value)
        self.cellsConnections[key-1] = edgeList            
                
                
    def defineGridGraph(self,rows,cols):
        self.cellsConnections =[None for x in range(rows*cols) ]
        # Defining the edges between the cells
        # The order of traversing in grid will be in this order
        # [Up-Right-Down-Left]
        for x in self.cells:
            self.connectEdge(x,x-cols,rows,cols)
            self.connectEdge(x,x+1,rows,cols)
            self.connectEdge(x,x+cols,rows,cols)
            self.connectEdge(x,x-1,rows,cols)
        return
    
    def findShortest(self):
    # This function is finding the nearest accessible object or goal from the robot
        shortestPath=[]
        if (self.EmptyRobotArm):
            #Looping for all objects
            for x in self.objects:
                #Get the path for each object and save number of steps to each object
                path=self.BFS(self.Robot,x)
                   # continue
                if(path==None):
                    return
                path=path[1]
                shortestPath.append(len(path))
            #Then get the object with minimum steps and return it
            indexOfShortest=shortestPath.index(min(shortestPath))
            return self.objects[indexOfShortest]
        else:
            for x in self.goals:
                #Get the path for each goal and save number of steps to each object
                path=self.BFS(self.Robot,x)
              # if(path==None):
                  #  continue
                if(path==None):
                    return
                path=path[1]
                shortestPath.append(len(path))
            #Then get the goal with minimum steps and return it
            indexOfShortest=shortestPath.index(min(shortestPath))
            return self.goals[indexOfShortest]
            
        
    
    # Function the get the shortest path from BFS , it requires start and goal and the parents of each node
    def get_Path(self,start,end,parent):
        # Stack to get the path from back to front
        stack=[]
        # Getting the path till find the start point
        while(end != start):
            stack.append(end)
            end=parent[end-1]
        #when the path raches the start , it add the start to the stack
        stack.append(start)
        #Then reverse the stack to be the right path and return it
        stack=stack[::-1]
        return stack

    def BFS(self,start,end):
        # First defining visited array and parent to get the shortest path
        visited =[False for i in range(rows *cols)]
        parent=[None for i in range(rows*cols)]
        #Then makeing the start node is visited and push it to queue
        queue=[start]
        visited[start-1]=True
        # Checking if the goal is the start
        if start==end:
            print('The start and end are the same')
            return
        while len(queue) >0:
        # Pop first element in the queue
            node=queue.pop(0)
            if node==None:
                return None
            if self.cellsConnections[node-1]==None:
                return None
            # Getting the neighbours of each node , due to seted directions [Up-Right-Down-Left]
            for nextNode in self.cellsConnections[node-1]:
                #Check if node is visited do nothing
                if(visited[nextNode-1]==False):
                    # Intializing the current visited node to True
                    visited[nextNode-1]=True
                    # And set the parent of it
                    parent[nextNode-1]=node
                    #Then append the node to queue to be seached
                    queue.append(nextNode)
                    #finally when the goal point is reached , no need to traverse then find path and stop 
                    if nextNode==end:
                        path=self.get_Path(start,end,parent)
                        # Draw cellsGrid
                        # Draw the path
                        #Get visited Nodes by get indices of trues
                        visited=[i+1 for i, x in enumerate(visited) if x]
                        return [visited,path]
        return None
    
    def Delay(self,root,t):
        root.update()
        time.sleep(t)
            
    def finishDrawing(self,B1,B2):
        if B1['state'] ==tk.DISABLED:
            return
        if len(self.objects)< 1 or len(self.goals)<1 or self.Robot==None:
            messagebox.showinfo("Components Alert", "Choose Robot, Objects and Goals first")
            return
        if len(self.objects)!=len(self.goals):
            messagebox.showinfo("Components Alert", "Number of Objects and Goals are not equal")
            return
        self.draw=False
        B1['state']=tk.DISABLED
        B2['state']=tk.NORMAL
        
    def Reset(self,c,B1,B2,B3,label):
        if B3['state'] ==tk.DISABLED:
            return
        self.draw=True
        B1['state']=tk.NORMAL
        B2['state']=tk.DISABLED
        B3['state']=tk.DISABLED
        for x in self.cells:
            c.itemconfigure(x, fill="white")
        self.Robot=None
        self.EmptyRobotArm=True
        self.objects= []
        self.objectsAfterReachGoal=[]
        self.goals=[]
        self.obstacles=[]
        self.draw=True
        label['text']="Program Current Opeartion\n\n\nNot Started\n\n\n"
        
        
                    
    def robotAction(self,rows,cols,root,c,B2,B3,label):
        # Checking if the user still drawing the objects
        if self.draw==True:
            return
        #Then implementing the searc
        B2['state']=tk.DISABLED
        # First draw graph and edges of it
        objectNum=1
        while len(self.objects)>0 :
            self.defineGridGraph(rows,cols)
            # Set the start to robot State
            start=self.Robot
            #Set the end to nearest accessible Object state
            end=self.findShortest()
            label['text']="Program Current Opeartion\n\n\nSearching for\n nearest object "+str(objectNum)+" to pick\n\n"
            self.Delay(root,2)
            output=self.BFS(start,end)
            if(output==None):
                messagebox.showinfo("Components Alert", 'There is an object that is not reachable ,Reset and Delete some blocks to reach it')
                B3['state']=tk.NORMAL
                return
            else:
                # Then drawing the vistied nodes
                label['text']="Program Current Opeartion\n\n\nThen Drawing Visited \n nodes using BFS\n\n"
                for visited in output[0]:
                    if visited==start or visited==end or visited in self.goals or visited in self.objects or visited in self.objectsAfterReachGoal:
                        continue
                    c.itemconfigure(visited, fill="red")
                self.Delay(root,2)
                # Then Removing the visited nodes
                for visited in output[0]:
                    if visited==start or visited==end or visited in self.objects or visited in self.goals or visited in self.objectsAfterReachGoal:
                        continue
                    c.itemconfigure(visited, fill="white")
                label['text']="Program Current Opeartion\n\n\nThen Getting Shortest \n path to object "+str(objectNum)+"\n\n"
                # Then drawing the shortest path
                for visited in output[1]:
                    if visited==start or visited==end or visited in self.goals or visited in self.objects or visited in self.objectsAfterReachGoal:
                        continue
                    if visited not in self.goals:
                        c.itemconfigure(visited, fill="red")
                        self.Delay(root,0.2)
                #Then robot moving to the the cell near to object
                label['text']="Program Current Opeartion\n\n\nThen Moving to Object "+str(objectNum)+" \n \n\n"
                self.Delay(root,0.8)
                c.itemconfigure(self.Robot,fill='white')
                for visited in output[1][1:-1]:
                    if visited in self.goals:
                        c.itemconfigure(self.Robot,fill='gold')
                        #c.itemconfigure(visited, fill="green")
                        self.Robot=visited
                        self.Delay(root,0.2)
                    else:
                        c.itemconfigure(self.Robot,fill='white')
                        c.itemconfigure(visited, fill="green")
                        self.Robot=visited
                        self.Delay(root,0.2)
                self.Delay(root,0.2)
                #Then the robot holded the object
                label['text']="Program Current Opeartion\n\n\nThen Holding Object "+str(objectNum)+" \n \n\n"
                self.EmptyRobotArm=False
                # Saving the id of the holdedObject
                holdedObject=output[1][-1]
                c.itemconfigure(holdedObject,fill='white')
                # and remove it from current objects
                self.objects.remove(holdedObject)
                self.Delay(root,0.2)
                #Then the robot get the nearest accessible goal
                label['text']="Program Current Opeartion\n\n\nThen Finding Nearest Goal \n  \n\n"
                start=self.Robot
                end=self.findShortest()
                output=self.BFS(start,end)
                if(output==None):
                    messagebox.showinfo("Alert", 'There is an object that is not reachable please delete some blocks')
                    return
                else:
                    # Then drawing the vistied nodes
                    label['text']="Program Current Opeartion\n\n\nThen Drawing visited \n nodes using BFS \n\n"
                    for visited in output[0]:
                        if visited==start or visited==end or visited in self.objects or visited in self.goals or visited in self.objectsAfterReachGoal:
                            continue
                        c.itemconfigure(visited, fill="red")
                    self.Delay(root,2)
                     # Then Removing the visited nodes
                    for visited in output[0]:
                        if visited==start or visited==end or visited in self.objects or visited in self.goals or visited in self.objectsAfterReachGoal: 
                            continue
                        c.itemconfigure(visited, fill="white")
                    label['text']="Program Current Opeartion\n\n\nThen Getting Shortest \n path to Goal "+str(objectNum)+"\n\n"
                    # Then drawing the shortest path
                    for visited in output[1]:
                        if visited==start or visited==end or visited in self.goals or visited in self.objects or visited in self.objectsAfterReachGoal:
                            continue
                        if visited not in self.goals:
                            c.itemconfigure(visited, fill="red")
                            self.Delay(root,0.2)
                    # Then moving to the nearest goal
                    self.Delay(root,0.2)
                    label['text']="Program Current Opeartion\n\n\nThen Moving to Goal "+str(objectNum)+" \n \n\n"
                    c.itemconfigure(self.Robot,fill='white')
                    for visited in output[1][1:-1]:
                        if visited in self.goals:
                            c.itemconfigure(self.Robot,fill='gold')
                          #  c.itemconfigure(visited, fill="green")
                            self.Robot=visited
                            self.Delay(root,0.2)
                        else:
                            c.itemconfigure(self.Robot,fill='white')
                            c.itemconfigure(visited, fill="green")
                            self.Robot=visited
                            self.Delay(root,0.2)
                    self.Delay(root,0.2)
                    label['text']="Program Current Opeartion\n\n\nThen releasing object "+str(objectNum)+"\n \n\n"
                    #After reaching near to the goal , release the object
                    goalLocation=output[1][-1]
                    self.goals.remove(goalLocation)
                    c.itemconfigure(goalLocation,fill='blueviolet')
                    self.objectsAfterReachGoal.append(goalLocation)
                    self.EmptyRobotArm=True
                    objectNum+=1
        B3['state']=tk.NORMAL

                

        
    
    def Main(self,rows,cols):
        # Creating instance of tkinter
        root = tk.Tk()
        root.title('Robotic System')
        #Making the window not resizable
        root.resizable(False, False)
        # Creating the window 
        c = tk.Canvas(root, width=900, height=768, borderwidth=5, background='white',highlightthickness=2)
        c.pack(side='left')
        root.update()
        c2 = tk.Canvas(root,height=768, borderwidth=5, background='#EEEEEE',highlightthickness=2)
        c2.pack(side='right')
        # Calling the function that draw the cells
        self.buildCells(c,rows,cols)
        
        # Calling function that interpets when the key is clicked
        c.bind("<KeyPress>",lambda e: self.middle(event=e,c=c))
        c.focus_set()
        #Defining the user buttons
        B1 = tk.Button(c2, text ="FinishDrawing",width=20)
        B2 = tk.Button(c2, text ="Start_Searching",width=20,state=tk.DISABLED)
        B3 = tk.Button(c2, text ="Reset",width=20,state=tk.DISABLED)
        B3.pack(side='bottom')
        B2.pack(side='bottom')
        B1.pack(side='bottom')
        B1.bind("<Button-1>",lambda e: self.finishDrawing(B1,B2))
        B2.bind("<Button-1>",lambda e:self.robotAction(rows,cols,root,c,B2,B3,label3))
        B3.bind("<Button-1>",lambda e:self.Reset(c,B1,B2,B3,label3))
        # Creating label to demonstrates the gui
        Instructions= "1.Press B and click to Add Obstacle (Gray) \n\n"\
                      "2.Press R and click to select Robot (Green) \n\n"\
                      "3.Press O and click to select Objects (Blue Violet )\n\n"\
                      "4.Press G and click to select Goals (Gold) \n\n"\
                      "5.Press D To delete Objects \n\n"\
                      "6.After selecting all components\n"\
                       " press on Finish Drawing \n\n"\
                        "7. Then click on Start Searching \n\n"\
                        "8.After, finishing click reset to start again\n"
                      
        label1 =Label (c2,text="Instructions \n ",font='Helvetica 14 bold')
        label2 =Label (c2, text=Instructions,font='Helvetica 12 bold')
        label3 =Label (c2,text="Program Current Opeartion\n\n\nNot Started\n\n\n",font='Helvetica 16 bold',fg='red')
        label3.pack(side='top')
        label1.pack(side="top")
        label2.pack(side="top")
        # Run the winodw
        root.mainloop()

    

## Implementation an instance of the class 

In [3]:
#Defining number of rows and cols
rows=10
cols=12
x=roboticSystem()
x.Main(rows,cols)